# Binary Classification with Logistic Regression

Use a binary classification dataset and train a logistic regression model. Classify the dataset using the trained model and print out the rows of the first class. Save the logistic regression model to logistic_regression.mlnet. Once completed, print 'task resolved' in the end.

## Binary Classification with Logistic Regression

Use a binary classification dataset and train a logistic regression model. Classify the dataset using the trained model and print out the rows of the first class. Save the logistic regression model to logistic_regression.mlnet. Once completed, print 'task resolved' in the end.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

var mlContext = new MLContext(seed: 0);

var dataPoints = GenerateRandomDataPoints(1000);

var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

var pipeline = mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression();

var model = pipeline.Fit(trainingData);

var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));

var transformedTestData = model.Transform(testData);

var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();

foreach (var p in predictions.Where(p => p.PredictedLabel))
{
    Console.WriteLine($"Label: {p.Label}, Prediction: {p.PredictedLabel}");
}

mlContext.Model.Save(model, trainingData.Schema, "logistic_regression.mlnet");

Console.WriteLine("task resolved");

IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.5f;
        yield return new DataPoint
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50).Select(x => x ? randomFloat() : randomFloat() + 0.1f).ToArray()
        };
    }
}

class DataPoint
{
    public bool Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

class Prediction
{
    public bool Label { get; set; }
    public bool PredictedLabel { get; set; }
}